In [1]:
!pip install mahaNLP==0.5 gdown -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00


In [2]:
!gdown --id 199Eo_tf8_7U-CPjVm74F4ijk-8lahpQh

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=199Eo_tf8_7U-CPjVm74F4ijk-8lahpQh
To: /content/goemotions_subset_train_new_part1.csv
100% 3.44M/3.44M [00:00<00:00, 265MB/s]


In [3]:
# importing the necessary libraries
import mahaNLP
import pandas as pd
import torch
import random
import re
import gdown

In [4]:
from mahaNLP.tokenizer import Tokenize
tokenizer = Tokenize()
from mahaNLP.tagger import EntityRecognizer
ner_tagger = EntityRecognizer()
from mahaNLP.maskFill import MaskPredictor
mask_predictor = MaskPredictor()

In [5]:
import re

# preprocessing function, don't remove the punctuations at all.
def remove_non_marathi_goemotions(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
#     i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [6]:
df = pd.read_csv('/content/goemotions_subset_train_new_part1.csv')
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,three_point,marathi_text
0,0,0,13178,"This is so hard to sit through, I can't believe the swearing from this dood, it makes it all the more unbearable",-1,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते"
1,1,1,39432,Because idiots in this sub care too much about what other people have on their screen? It only tells me that too many people on here are narcissists.,-1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.
2,3,3,30127,Water is not a human rights. That's absurd.,-1,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.
3,4,4,10283,The thin side and endlines are really throwing me off.,-1,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.
4,5,5,2930,If there is no goal on this powerplay it will be difficult not to be hopeless,-1,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल


In [7]:
df.drop(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'text'], inplace=True, axis=1)
dict1 = {'marathi_text': 'tweet', 'three_point': 'label'}
df.rename(columns=dict1, inplace=True)
df = df.iloc[:, [1, 0]]

In [8]:
df.head()

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल,-1


In [9]:
for i in range(len(df)):
  df['tweet'][i] = remove_non_marathi_goemotions(df['tweet'][i])

<ipython-input-9-64eed04c3b0f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i] = remove_non_marathi_goemotions(df['tweet'][i])


In [10]:
df = df[df['tweet']!='']

In [11]:
df.head()

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल,-1


In [12]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True)

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल,-1
...,...,...
12486,स्त्रियांना पुरुषांसारखेच सर्व अधिकार मिळाले पाहिजेत. त्यात दुसऱ्या माणसाचे जीवन संपवणे समाविष्ट नाही.,1
12487,या व्यक्तीकडे वस्तुस्थितीनंतर काही मोठ्या प्रमाणात आत्मनियंत्रण आहे. मला परत आल्यासारखं वाटलं आणि त्या क्रोकला ठोसा मारावा.,1
12488,बुडणाऱ्या स्कूल बससारखे मजेदार.,1
12489,सर्वोत्तम ठिकाणे स्थलांतरित रेस्टॉरंट्स आहेत. स्वस्त आणि स्वादिष्ट अन्न. जे चांगले आहेत ते डाउनटाउनच्या बाहेर आहेत.,1


In [13]:
len(df)

12491

In [15]:
df['tweet'][1022]

'दुसर्\u200dया कीमनसाठी कीमुनपासून सुटका करण्यात अर्थ नाही.'

In [23]:
re.sub(r'[\u200d]+', '', df['tweet'][1022])
# i=re.split(r'[\s\n]+', i.strip())

'दुसर्या कीमनसाठी कीमुनपासून सुटका करण्यात अर्थ नाही.'

In [25]:
def mask_NER_entities_in_text_sequential(text):
    # getting the NER tokens from the sentence
    df_ner = ner_tagger.getTokenLabels(text)
    

    # converting the NER tokens into a list, excluding those from the 'Other' category
    ner_tokens = list(df_ner.loc[df_ner['entity_group'] != 'Other', 'word'])
    # print('ner_tokens', ner_tokens)


    sentence=text
    for i in ner_tokens:
        # replacing the first occurance of the NER token with [MASK]
        temp = sentence.replace(i, '[MASK]', 1)
        # print(temp)

        # getting the prediction with the highest score, and replacing the sentence with that
        if(mask_predictor.predictMask(temp)['token_str'][0]!=i):
          sentence = mask_predictor.predictMask(temp)['sequence'][0]
        else:
          sentence = mask_predictor.predictMask(temp)['sequence'][1]

    return(sentence)

In [26]:
df['tweet'][5]

'मला हे स्पष्ट करायचे आहे की मी घाबरत नाही. मी आरशावर ठोसा मारला कारण तो माझ्याकडे चुकीचा दिसत होता.'

In [27]:
mask_NER_entities_in_text_sequential(df['tweet'][5])

'मला हे स्पष्ट करायचे आहे की मी घाबरत नाही. मी त्याला ठोसा मारला कारण तो माझ्याकडे चुकीचा दिसत होता.'

In [28]:
list_augmented_sentences = []

In [29]:
df_augmented = pd.DataFrame(columns=df.columns)

In [31]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    j = re.sub(r'[\u200d]+', '', row['tweet'])
    tweet_augmented = mask_NER_entities_in_text_sequential(j)
    row_augmented = row.copy()
    row_augmented['tweet_augmented'] = tweet_augmented
    df_augmented = df_augmented.append(row_augmented, ignore_index=True)

df_augmented.drop(columns=['tweet'], axis=1, inplace=True)

Streaming output truncated to the last 5000 lines.
 80%|███████▉  | 9992/12491 [49:16<09:56,  4.19it/s]<ipython-input-31-01b166dc4bfc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_augmented = df_augmented.append(row_augmented, ignore_index=True)
 80%|████████  | 9993/12491 [49:16<09:07,  4.56it/s]<ipython-input-31-01b166dc4bfc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_augmented = df_augmented.append(row_augmented, ignore_index=True)
 80%|████████  | 9994/12491 [49:17<08:22,  4.97it/s]<ipython-input-31-01b166dc4bfc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_augmented = df_augmented.append(row_augmented, ignore_index=True)
 80%|████████  | 9995/12491 [49:17<08:00,  5.20it/s]<ipython-input-31-01b166

In [32]:
df_augmented = df_augmented.rename(columns={'tweet_augmented': 'tweet'})


<ipython-input-32-09d8652a5a0a>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df.append(df_augmented, ignore_index = True)


In [39]:
df_augmented =df_augmented.iloc[:, [1, 0]]

In [41]:
df

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल,-1
...,...,...
12496,स्त्रियांना पुरुषांसारखेच सर्व अधिकार मिळाले पाहिजेत. त्यात दुसऱ्या माणसाचे जीवन संपवणे समाविष्ट नाही.,1
12497,या व्यक्तीकडे वस्तुस्थितीनंतर काही मोठ्या प्रमाणात आत्मनियंत्रण आहे. मला परत आल्यासारखं वाटलं आणि त्या क्रोकला ठोसा मारावा.,1
12498,बुडणाऱ्या स्कूल बससारखे मजेदार.,1
12499,सर्वोत्तम ठिकाणे स्थलांतरित रेस्टॉरंट्स आहेत. स्वस्त आणि स्वादिष्ट अन्न. जे चांगले आहेत ते डाउनटाउनच्या बाहेर आहेत.,1


In [53]:
df_augmented

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल,-1
...,...,...
12486,स्त्रियांना पुरुषांसारखेच सर्व अधिकार मिळाले पाहिजेत. त्यात दुसऱ्या माणसाचे जीवन संपवणे समाविष्ट नाही.,1
12487,या व्यक्तीकडे वस्तुस्थितीनंतर काही मोठ्या प्रमाणात आत्मनियंत्रण आहे. मला परत आल्यासारखं वाटलं आणि त्या क्रोकला ठोसा मारावा.,1
12488,बुडणाऱ्या स्कूल बससारखे मजेदार.,1
12489,सर्वोत्तम ठिकाणे स्थलांतरित रेस्टॉरंट्स आहेत. स्वस्त आणि स्वादिष्ट अन्न. जे चांगले आहेत ते डाउनटाउनच्या बाहेर आहेत.,1


In [56]:
# df = pd.concat([df, df_augmented])
df_final = df.append(df_augmented, ignore_index = True)
df_final.drop_duplicates(inplace=True)
df_final.dropna(inplace=True)
df_final=df_final.reset_index(drop=True)
df_final.to_csv('goemotions_sequential_masking_augmented_1.csv', index=False)

<ipython-input-56-8cbd91a974ad>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df.append(df_augmented, ignore_index = True)


In [55]:
df_final

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न होणे कठीण होईल,-1
...,...,...
14531,"मला डाउनव्होट आणि त्यानंतरचा प्रतिसाद आवडतो. धन्यवाद स्त्रिया आणि सज्जनांनो, शुभे * माईक ड्रॉप",1
14532,मी १० : ३० आणि माझे हास्य आता घरघरात रुपांतरीत झाले आहे,1
14533,मधू गोंडस आहे! माझ्या मावशी आणि काकांकडे हनी नावाचा कुत्राही आहे.,1
14534,माझा आवडता नवीन चाहता! तुम्हाला येथे पाहून आनंद झाला! आराम करा आणि या खेळासाठी सज्ज व्हा.,1


In [49]:
len(df_final)

14536

In [35]:
len(df_augmented)

12491

In [50]:
len(df)

12491

In [51]:
# 14536-12491 = 2045 new sentences formed.